# emTools

### a Python module to interact with SerialEM to enable automated Transmission Electron Microscopy

## Installation instructions

### Prepare your PC for the use of Python and emTools

- first we will install Python and then use it to fetch the packages needed for emTools to work.
- Install Anaconda from [here](https://www.anaconda.com/download/ "Download Anaconda"). EmTools has been tested to work with both Python 2 and 3. 
- Start the Anaconda [Navigator](http://docs.anaconda.com/anaconda/user-guide/getting-started/ "Getting started with Anaconda")
- as explained in [section 4](http://docs.anaconda.com/anaconda/user-guide/getting-started/#run-python-in-a-jupyter-notebook "Run Python in a Jupyter Notebook"), install and start a Jupyter notebook.
- Download this [tutorial file](https://git.embl.de/schorb/serialem_tools/raw/master/emTools.ipynb) (use "save link as..."), open it in you Jupyter session and follow the instructions.


----
### How to use this jupyter notebook

In this document there are text blocks and code blocks that can be edited and executed.
In order to run a code block, click into it to activate and then press CTRL+Enter.


### Install necessary dependencies in Python

The following code block uses Anaconda to fetch necessary packages

In [ ]:
import conda.cli

conda.cli.main('conda', 'install',  '-y', 'numpy', 'scipy', 'pandas', 'scikit-image', 'git')
conda.cli.main('conda', 'install',  '-y', '-c conda-forge', 'tifffile')

Now we will install emTools using pip, another package manager for Python.

In [2]:
from pip._internal import main as pipmain

pipmai (['install','git+https://git.embl.de/schorb/serialem_tools'])

AttributeError: 'module' object has no attribute '_internal'